In [26]:

# imports
import time
import copy
import pandas as pd
import numpy as np
import itertools as it
import seaborn as sns
import matplotlib.pyplot as plt

from collections import defaultdict, Counter
from pprint import pprint

from sklearn.metrics import accuracy_score
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.exceptions import NotFittedError

from pmlb import fetch_data


In [27]:
# functions
def timer(func):
    def wrapper(*args, **kwargs):
        start = time.perf_counter()
        result = func(*args, **kwargs)
        end = time.perf_counter()
        print(f"{func.__name__} took {end - start:.4f} seconds")
        return result
    return wrapper

def create_classifier(X, y, ignored_features=[]):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)
    X_train = X_train.drop(columns=ignored_features)
    X_test = X_test.drop(columns=ignored_features)
    
    numeric_features = X_train.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X_train.select_dtypes(include=['category', 'object']).columns
    print(numeric_features, categorical_features)
    
    numeric_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('scaler', StandardScaler())])
    categorical_transformer = Pipeline(steps=[('imputer', SimpleImputer(strategy='most_frequent')), ('onehot', OneHotEncoder(handle_unknown='ignore'))])
    
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),
            ('cat', categorical_transformer, categorical_features)
        ],
        remainder='passthrough'
    )
    
    clf = Pipeline(steps=[('preprocessor', preprocessor),
                          ('classifier', DecisionTreeClassifier(random_state=0))])
    
    param_grid = {
        'classifier__max_depth': [5, 10, 20, None],
        'classifier__min_samples_split': [2, 5, 10, 20],
        'classifier__min_samples_leaf': [1, 5, 10, 20],
        'classifier__criterion': ['gini', 'entropy']
    }
    grid = GridSearchCV(clf, param_grid, cv=5, n_jobs=-1)
    grid.fit(X_train, y_train)
    clf = grid.best_estimator_
    accuracy = accuracy_score(y_test, clf.predict(X_test))
    
    return clf, accuracy

def get_feature_names(clf):
    feature_names = []
    for name, transformer, features in clf.named_steps['preprocessor'].transformers_:
        if name == 'num':
            feature_names.extend(features)
        elif name == 'cat':
            try:
                feature_names.extend(transformer.named_steps['onehot'].get_feature_names_out(features))
            except NotFittedError:
                print("Warning: no categorical features found")
    return feature_names
    
def calculate_fairness_metrics(clf, X, y, sensitive_attributes, bad_outcome, ignored_features=[]):
    X_train, X_test_full, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)
    X_test = X_test_full.drop(columns=ignored_features)

    good_outcome = not bad_outcome
    value_combinations = [*it.product(*[X[sensitive_attribute].unique() for sensitive_attribute in sensitive_attributes])]
    metrics = {'TP': {}, 'FP': {}, 'TN': {}, 'FN': {}, 'PPV': {}, 'PR': {}, 'TPR': {}, 'FPR': {}, 'acc': {}, 'balanced_acc': {}}
    
    for attribute in value_combinations:
        for key in ['TP', 'FP', 'TN', 'FN']:
            metrics[key][attribute] = 0

    for idx in range(len(y_test)):
        attribute = tuple(X_test_full.iloc[idx][sensitive_attributes])

        prediction = clf.predict(X_test.iloc[idx:idx + 1, :])[0]
        actual = y_test.iloc[idx]

        if prediction == good_outcome:
            if actual == good_outcome:
                metrics['TP'][attribute] += 1
            else:
                metrics['FP'][attribute] += 1
        elif prediction == bad_outcome:
            if actual == bad_outcome:
                metrics['TN'][attribute] += 1
            else:
                metrics['FN'][attribute] += 1

    for attribute in value_combinations:
        TP = metrics['TP'][attribute]
        FP = metrics['FP'][attribute]
        TN = metrics['TN'][attribute]
        FN = metrics['FN'][attribute]
        
        try:
            metrics['PPV'][attribute] = TP / (TP + FP)  # Positive Predictive Value
            metrics['PR'][attribute] = (TP + FP) / (TP + FP + TN + FN)  # Precision-Recall
            metrics['TPR'][attribute] = TP / (TP + FN)  # True Positive Rate
            metrics['FPR'][attribute] = FP / (FP + TN)  # False Positive Rate
            metrics['balanced_acc'][attribute] = 0.5 * (TP / (TP + FN) + TN / (TN + FP))  
            metrics['acc'][attribute] = (TP + TN) / (TP + FP + TN + FN)  
        except ZeroDivisionError:
            for key in ['PPV', 'PR', 'TPR', 'FPR', 'balanced_acc', 'acc']:
                metrics[key][attribute] = 0

    for attribute in value_combinations:
        print(f"Group: {attribute}")
        for metric_name, metric_values in metrics.items():
            print(f"  {metric_name}: {metric_values[attribute]}")
        print()

    return metrics['PPV'], metrics['PR'], metrics['TPR'], metrics['FPR'], metrics['acc'], metrics['balanced_acc']

def calculate_explicit_bias(clf, X, y, sensitive_attributes, bad_outcome):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)
    value_combinations = [*it.product(*[X[sensitive_attribute].unique() for sensitive_attribute in sensitive_attributes])]
    factual, explanation = [], []

    for idx in range(len(y_test)):
        to_explain = X_test.iloc[idx:idx+1, :]  
        
        if clf.predict(to_explain)[0] == bad_outcome:
            CF = copy.deepcopy(to_explain)  

            # print(f'Instance {idx}', value_combinations)
            for values in value_combinations:
                # print(CF, values)
                CF[sensitive_attributes] = values
                # print(CF, clf.predict(CF)[0], bad_outcome)
                if clf.predict(CF)[0] != bad_outcome:
                    factual_changes = []
                    explanation_changes = []

                    for attr, original, counterfactual in zip(sensitive_attributes, to_explain.iloc[0][sensitive_attributes], values):
                        # print(attr, original, counterfactual, original != counterfactual)
                        if original != counterfactual:
                            factual_changes.append(original)
                            explanation_changes.append(counterfactual)

                    factual.append(str(factual_changes))
                    explanation.append(str(explanation_changes))
                    break  

    factual_dict = Counter(factual)
    explanation_dict = Counter(explanation)

    print(f'Explicit bias detected:\nFactual Changes: {factual_dict}\nExplanation Changes: {explanation_dict}')

    return factual_dict, explanation_dict


def plot_feature_importance(clf, sensitive_attributes, filename):
    feature_importances = clf['classifier'].feature_importances_
    feature_names = get_feature_names(clf)
    
    feature_data = pd.DataFrame(zip(feature_names, feature_importances), columns=["feature", "value"])
    feature_data["abs_value"] = feature_data["value"].abs()
    feature_data["color"] = feature_data["value"].apply(lambda x: "green" if x > 0 else "red")
    feature_data = feature_data.sort_values("abs_value", ascending=False)
    
    fig, ax = plt.subplots(figsize=(12, 7))
    sns.barplot(x="feature", y="value", data=feature_data.head(10), palette='mako', ax=ax)
    
    ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=14)
    ax.set_title("Top 10 Features", fontsize=20)
    ax.set_xlabel("Feature Name", fontsize=16)
    ax.set_ylabel("Feature Importance", fontsize=16)
    plt.suptitle(f'Sensitive Attributes: {sensitive_attributes}', fontsize=18)
    
    fig.savefig(f"{filename}_feature_importance.png", bbox_inches='tight')
    plt.close(fig)
    
def plot_explicit_bias(sensitive_attributes, factual_dict, explanation_dict, filename):
    fig, ax = plt.subplots()
    ax.bar(factual_dict.keys(), factual_dict.values(), label=f'Counterfactual values', color='red')
    ax.set_title(f"Explicit bias with {', '.join(sensitive_attributes)}", fontsize=12)
    ax.tick_params(axis='x', rotation=90)
    fig.savefig(f"{filename}_explicit_bias_counterfactual.png", bbox_inches='tight')
    plt.close(fig)
    
    fig, ax = plt.subplots()
    ax.bar(explanation_dict.keys(), explanation_dict.values(), label=f'Factual values', color='green')
    ax.set_title(f"Explicit bias with {', '.join(sensitive_attributes)}", fontsize=12)
    ax.tick_params(axis='x', rotation=90)
    fig.savefig(f"{filename}_explicit_bias_factual.png", bbox_inches='tight')
    plt.close(fig)

@timer
def get_counterfactual_counts(clf, X_test, multi_variable=False):    
    predictions = clf.predict(X_test)
    
    X_test_transformed = clf.named_steps['preprocessor'].transform(X_test)
    feature_names = get_feature_names(clf)
    original_feature_names = X_test.columns.tolist()

    counterfactual_counts = defaultdict(int, {feature: 0 for feature in original_feature_names})
        
    tree = clf.named_steps['classifier']
    node_indicator = tree.decision_path(X_test_transformed)
    leaf_id = tree.apply(X_test_transformed)
    
    
    for sample_id in range(X_test.shape[0]):
        target_class = not predictions[sample_id]
        node_index = node_indicator.indices[
            node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
        ]
        
        single_var_counterfactuals = set()
        multi_var_counterfactuals = set()
        
        for node_id in node_index:
            if leaf_id[sample_id] == node_id:
                continue
            
            feature_id = tree.tree_.feature[node_id]
            threshold = tree.tree_.threshold[node_id]
            
            feature_name = feature_names[feature_id]
            original_feature_name = max((name for name in original_feature_names if name in feature_name), key=len)
            feature_value = X_test_transformed[sample_id, feature_id]
            
            counterfactual = X_test_transformed[sample_id].copy()
            if hasattr(counterfactual, "toarray"):
                counterfactual = counterfactual.toarray()[0]
            
            if feature_value <= threshold:
                counterfactual[feature_id] = threshold + 0.01
            else:
                counterfactual[feature_id] = threshold - 0.01
            
            curr_prediction = tree.predict(counterfactual.reshape(1, -1))[0]
            
            if curr_prediction == target_class or multi_variable is False:
                if curr_prediction == target_class:
                    single_var_counterfactuals.add(original_feature_name)
                continue
            
            new_node_index = tree.decision_path(counterfactual.reshape(1, -1)).indices
            new_path_nodes = [node for node in new_node_index if node not in node_index]
            
            for new_node_id in new_path_nodes:
                new_feature_id = tree.tree_.feature[new_node_id]
                new_threshold = tree.tree_.threshold[new_node_id]
                
                if new_feature_id == -2 or new_feature_id == feature_id:
                    continue
                
                new_feature_name = feature_names[new_feature_id]
                new_original_feature_name = max((name for name in original_feature_names if name in new_feature_name), key=len)
                new_feature_value = counterfactual[new_feature_id]
                
                new_counterfactual = counterfactual.copy()
                if new_feature_value <= new_threshold:
                    new_counterfactual[new_feature_id] = new_threshold + 0.01
                else:
                    new_counterfactual[new_feature_id] = new_threshold - 0.01
                
                new_prediction = tree.predict(new_counterfactual.reshape(1, -1))[0]
                if new_prediction == target_class:
                    multi_var_counterfactuals.add(tuple(sorted([original_feature_name, new_original_feature_name])))
                
        for feature in single_var_counterfactuals | multi_var_counterfactuals:
            counterfactual_counts[feature] += 1
            
    return counterfactual_counts

@timer
def get_counterfactual_counts_old(clf, X_test, default_values):
    predictions = clf.predict(X_test)
    
    feature_names = X_test.columns
    counterfactual_counts = defaultdict(int, {feature: 0 for feature in feature_names})
        
    for sample_id in range(X_test.shape[0]):
        instance = X_test.iloc[sample_id:sample_id + 1, :]
        target_class = not predictions[sample_id]
        
        for feature in feature_names:
            for val in default_values.get(feature, []):
                modified_instance = copy.deepcopy(instance)
                modified_instance[feature] = val
                
                if clf.predict(modified_instance)[0] == target_class:
                    counterfactual_counts[feature] += 1
                    break
            
    return counterfactual_counts

@timer
def get_counterfactual_counts_descriptive(clf, X_test, multi_variable=False):    
    predictions = clf.predict(X_test)
    
    X_test_transformed = clf.named_steps['preprocessor'].transform(X_test)
    feature_names = get_feature_names(clf)
    original_feature_names = X_test.columns.tolist()
    
    counterfactual_counts = defaultdict(int)
    factual_changes = defaultdict(Counter)
    explanation_changes = defaultdict(Counter)
        
    tree = clf.named_steps['classifier']
    node_indicator = tree.decision_path(X_test_transformed)
    leaf_id = tree.apply(X_test_transformed)
    
    
    for sample_id in range(X_test.shape[0]):
        target_class = not predictions[sample_id]
        original_instance = X_test.iloc[sample_id]
        
        node_index = node_indicator.indices[
            node_indicator.indptr[sample_id] : node_indicator.indptr[sample_id + 1]
        ]
        
        single_var_counterfactuals = set()
        multi_var_counterfactuals = set()
        
        curr_factual_changes = defaultdict(list)
        curr_explanation_changes = defaultdict(list)
        
        for node_id in node_index:
            if leaf_id[sample_id] == node_id:
                continue
            
            feature_id = tree.tree_.feature[node_id]
            threshold = tree.tree_.threshold[node_id]
            
            feature_name = feature_names[feature_id]
            original_feature_name = max((name for name in original_feature_names if name in feature_name), key=len)
            feature_value = X_test_transformed[sample_id, feature_id]
            
            counterfactual = X_test_transformed[sample_id].copy()
            if hasattr(counterfactual, "toarray"):
                counterfactual = counterfactual.toarray()[0]
            
            if feature_value <= threshold:
                counterfactual[feature_id] = threshold + 0.01
            else:
                counterfactual[feature_id] = threshold - 0.01
            
            curr_prediction = tree.predict(counterfactual.reshape(1, -1))[0]
            
            if curr_prediction == target_class:
                single_var_counterfactuals.add(original_feature_name)
                
                original_value = original_instance.get(original_feature_name, None)
                explanation_value = f"{feature_name} {'>' if counterfactual[feature_id] > threshold else '<='} {threshold:.2f}"
                
                curr_factual_changes[original_feature_name].append(original_value)
                curr_explanation_changes[original_feature_name].append(explanation_value)
                
                continue
            
            if not multi_variable: 
                continue
            
            new_node_index = tree.decision_path(counterfactual.reshape(1, -1)).indices
            new_path_nodes = [node for node in new_node_index if node not in node_index]
            
            for new_node_id in new_path_nodes:
                new_feature_id = tree.tree_.feature[new_node_id]
                new_threshold = tree.tree_.threshold[new_node_id]
                
                if new_feature_id == -2 or new_feature_id == feature_id:
                    continue
                
                new_feature_name = feature_names[new_feature_id]
                new_original_feature_name = max((name for name in original_feature_names if name in new_feature_name), key=len)
                new_feature_value = counterfactual[new_feature_id]
                
                new_counterfactual = counterfactual.copy()
                if new_feature_value <= new_threshold:
                    new_counterfactual[new_feature_id] = new_threshold + 0.01
                else:
                    new_counterfactual[new_feature_id] = new_threshold - 0.01
                
                new_prediction = tree.predict(new_counterfactual.reshape(1, -1))[0]
                if new_prediction == target_class:
                    multi_var_counterfactuals.add(tuple(sorted([original_feature_name, new_original_feature_name])))
                    
                    original_values = (original_instance.get(original_feature_name, None),
                                      original_instance.get(new_original_feature_name, None))
                    
                    explanation_value = f"{feature_name} {'>' if counterfactual[feature_id] > threshold else '<='} {threshold:.2f}"
                    new_explanation_value = f"{new_feature_name} {'>' if new_counterfactual[new_feature_id] > new_threshold else '<='} {new_threshold:.2f}"

                    if sorted([original_feature_name, new_original_feature_name]) == [original_feature_name, new_original_feature_name]:
                        curr_factual_changes[tuple(sorted([original_feature_name, new_original_feature_name]))].append(original_values)
                        curr_explanation_changes[tuple(sorted([original_feature_name, new_original_feature_name]))].append((explanation_value, new_explanation_value))
                    else:
                        curr_factual_changes[tuple(sorted([new_original_feature_name, original_feature_name]))].append(original_values[::-1])
                        curr_explanation_changes[tuple(sorted([new_original_feature_name, original_feature_name]))].append((new_explanation_value, explanation_value))
                
        for feature in single_var_counterfactuals | multi_var_counterfactuals:
            counterfactual_counts[feature] += 1
            factual_changes[feature].update(curr_factual_changes[feature])
            explanation_changes[feature].update(curr_explanation_changes[feature])
            
    return counterfactual_counts, factual_changes, explanation_changes

@timer
def get_precof_values(clf, X, y, minority_class, minority_value, negative_outcome, method="new", multi_variable=False):
    X_test = train_test_split(X, test_size=0.3, stratify=y, random_state=0)[1]
    X_negative = X_test[clf.predict(X_test) == negative_outcome]
        
    X_minority = X_negative[X_negative[minority_class] == minority_value]
    X_majority = X_negative[X_negative[minority_class] != minority_value]
    
    minority_total = X_minority.shape[0]
    majority_total = X_majority.shape[0]
    
    if method == "new":
        if minority_total > 0:
            minority_counts = get_counterfactual_counts(clf, X_minority, multi_variable)
        if majority_total > 0:
            majority_counts = get_counterfactual_counts(clf, X_majority, multi_variable)
    elif method == "old":
        default_values = calculate_default_values(X, y)
        if minority_total > 0:
            minority_counts = get_counterfactual_counts_old(clf, X_minority, default_values)
        if majority_total > 0:
            majority_counts = get_counterfactual_counts_old(clf, X_majority, default_values)
    elif method == "descriptive":
        minority_factual_changes = majority_factual_changes = defaultdict(Counter)
        minority_explanation_changes = majority_explanation_changes = defaultdict(Counter)
        
        if minority_total > 0:
            minority_counts, minority_factual_changes, minority_explanation_changes = get_counterfactual_counts_descriptive(clf, X_minority, multi_variable)
        if majority_total > 0:
            majority_counts, majority_factual_changes, majority_explanation_changes = get_counterfactual_counts_descriptive(clf, X_majority, multi_variable)
            
    if majority_total == 0:
        majority_counts = {feature: 0 for feature in minority_counts.keys()}
    elif minority_total == 0:
        minority_counts = {feature: 0 for feature in majority_counts.keys()}
    
    precof_values = {}
    features = set(minority_counts.keys()).union(set(majority_counts.keys()))
    
    for feature in features:
        minority_value = minority_counts.get(feature, 0) / minority_total if minority_total > 0 else 0
        majority_value = majority_counts.get(feature, 0) / majority_total if majority_total > 0 else 0
        
        precof = minority_value - majority_value
        precof_values[feature] = precof
        
    if method != "descriptive":
        return precof_values
    else:    
        return precof_values, minority_factual_changes, minority_explanation_changes, majority_factual_changes, majority_explanation_changes

def calculate_default_values(X, y):
    X_train, *_ = train_test_split(X, y, test_size=0.3, stratify=y, random_state=0)
    
    numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
    categorical_features = X.select_dtypes(include=['category', 'object']).columns
    
    default_values = {}
    threshold = len(X_train) / 100
    
    for feature in numeric_features:
        default_values[feature] = [*{*np.percentile(X_train[feature], [10, 20, 30, 40, 50, 60, 70, 80, 90, 100])}]  # Deduplicate
    for feature in categorical_features:
        default_values[feature] = []
        value_counts = X_train[feature].value_counts()
        for val, count in value_counts.items():
            if count < threshold: break
            default_values[feature].append(val)
            
        if len(default_values[feature]) == 0 or len(default_values[feature]) > 10:
            default_values[feature] = value_counts.index[:10].tolist()
          
    return default_values

def full_precof_analysis(X, y, sensitive_attributes, minority_class, minority_value, negative_outcome, name):
    clf, accuracy = create_classifier(X, y)
    
    precof_values = get_precof_values(clf, X, y, minority_class, minority_value, negative_outcome, method="new")
    multi_var_precof_values = get_precof_values(clf, X, y, minority_class, minority_value, negative_outcome, method="new", multi_variable=True)
    old_precof_values = get_precof_values(clf, X, y, minority_class, minority_value, negative_outcome, method="old")
    fairness_metrics = calculate_fairness_metrics(clf, X, y, sensitive_attributes, negative_outcome)
    factual_dict, explanation_dict = calculate_explicit_bias(clf, X, y, sensitive_attributes, negative_outcome)

    plot_feature_importance(clf, sensitive_attributes, f"{name}_full")
    plot_explicit_bias(sensitive_attributes, factual_dict, explanation_dict, f"{name}_full")
    
    print("Model with sensitive attributes")
    print("=======================")
    print(f"Classifier Accuracy: {accuracy:.4f}\n")
    
    print(f"Fairness Metrics: {fairness_metrics}")
    print(f'Explicit bias detected:\nFactual Changes: {factual_dict}\nExplanation Changes: {explanation_dict}')
    print(f"PreCoF Values: {sorted(precof_values.items(), key=lambda x: x[1], reverse=True)}")
    print(f"Multi-Variable PreCoF Values: {sorted(multi_var_precof_values.items(), key=lambda x: x[1], reverse=True)}")
    print(f"Old PreCoF Values: {sorted(old_precof_values.items(), key=lambda x: x[1], reverse=True)}")
    
    clf, accuracy = create_classifier(X, y, ignored_features=sensitive_attributes)

    precof_values = get_precof_values(clf, X, y, minority_class, minority_value, negative_outcome, method="new")
    multi_var_precof_values = get_precof_values(clf, X, y, minority_class, minority_value, negative_outcome, method="new", multi_variable=True)
    old_precof_values = get_precof_values(clf, X, y, minority_class, minority_value, negative_outcome, method="old")
    fairness_metrics = calculate_fairness_metrics(clf, X, y, sensitive_attributes, negative_outcome)
    
    plot_feature_importance(clf, sensitive_attributes, f"{name}_no_sensitive")
    
    print("\nModel without sensitive attributes")
    print("=======================")
    print(f"Classifier Accuracy: {accuracy:.4f}\n")
    
    print(f"Fairness Metrics: {fairness_metrics}")
    print(f"PreCoF Values: {sorted(precof_values.items(), key=lambda x: x[1], reverse=True)}")
    print(f"Multi-Variable PreCoF Values: {sorted(multi_var_precof_values.items(), key=lambda x: x[1], reverse=True)}")
    print(f"Old PreCoF Values: {sorted(old_precof_values.items(), key=lambda x: x[1], reverse=True)}")

def precof_analysis(X, y, sensitive_attributes, minority_class, minority_value, negative_outcome, name):
    clf, accuracy = create_classifier(X, y, ignored_features=sensitive_attributes)

    precof_values, minority_factual_changes, minority_explanation_changes, majority_factual_changes, majority_explanation_changes = get_precof_values(clf, X, y, minority_class, minority_value, negative_outcome, method="descriptive", multi_variable=True)
    
    print("\nModel without sensitive attributes")
    print("=======================")
    print(f"Classifier Accuracy: {accuracy:.4f}\n")
    
    def print_changes(changes, top_precof=3, top_n=2):
        sorted_changes = sorted(changes.items(), key=lambda x: precof_values[x[0]], reverse=True)
        for i, (feature, values) in enumerate(sorted_changes[:min(top_precof, len(sorted_changes))]):
            print(f"Rank {i + 1} | {feature} (PreCoF: {precof_values[feature]:.4f}): {values.most_common()[:(min(top_n, len(values)))]}")
        for i, (feature, values) in enumerate(sorted_changes[-min(top_precof, len(sorted_changes)):]):
            print(f"Rank -{min(top_precof, len(sorted_changes)) - i} | {feature} (PreCoF: {precof_values[feature]:.4f}): {values.most_common()[:(min(top_n, len(values)))]}")
            
    print(f"Multi-Variable PreCoF Values:")
    pprint(sorted(precof_values.items(), key=lambda x: x[1], reverse=True))
    
    print("Minority Factual Changes: ")
    print_changes(minority_factual_changes)
    print("Minority Explanation Changes: ")
    print_changes(minority_explanation_changes)
    print("Majority Factual Changes: ")
    print_changes(majority_factual_changes)
    print("Majority Explanation Changes: ")
    print_changes(majority_explanation_changes)


In [28]:
student = pd.read_csv('DATA/student/student-por.csv', sep=';')

X = student.drop(columns=['G1', 'G2', 'G3'])  
y = student['G3'] >= student['G3'].mean()  

full_precof_analysis(X, y, ['sex', 'guardian'], 'sex', 'F', False, 'student_por')

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'],
      dtype='object') Index(['school', 'sex', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob',
       'reason', 'guardian', 'schoolsup', 'famsup', 'paid', 'activities',
       'nursery', 'higher', 'internet', 'romantic'],
      dtype='object')
get_counterfactual_counts took 0.0214 seconds
get_counterfactual_counts took 0.0148 seconds
get_precof_values took 0.0424 seconds
get_counterfactual_counts took 0.0574 seconds
get_counterfactual_counts took 0.0352 seconds
get_precof_values took 0.0982 seconds
get_counterfactual_counts_old took 20.2563 seconds
get_counterfactual_counts_old took 11.9696 seconds
get_precof_values took 32.2409 seconds
Group: ('F', 'mother')
  TP: 37
  FP: 6
  TN: 20
  FN: 15
  PPV: 0.8604651162790697
  PR: 0.5512820512820513
  TPR: 0.7115384615384616
  FPR: 0.23076923076923078
  acc: 0.7307692307692307
  balanced_acc: 0.740

In [29]:
student = pd.read_csv('DATA/student/student-por.csv', sep=';')

X = student.drop(columns=['G1', 'G2', 'G3'])  
y = student['G3'] >= student['G3'].mean()  

precof_analysis(X, y, ['sex', 'guardian'], 'sex', 'F', False, 'student_por')

Index(['age', 'Medu', 'Fedu', 'traveltime', 'studytime', 'failures', 'famrel',
       'freetime', 'goout', 'Dalc', 'Walc', 'health', 'absences'],
      dtype='object') Index(['school', 'address', 'famsize', 'Pstatus', 'Mjob', 'Fjob', 'reason',
       'schoolsup', 'famsup', 'paid', 'activities', 'nursery', 'higher',
       'internet', 'romantic'],
      dtype='object')
get_counterfactual_counts_descriptive took 0.0747 seconds
get_counterfactual_counts_descriptive took 0.0426 seconds
get_precof_values took 0.1229 seconds

Model without sensitive attributes
Classifier Accuracy: 0.6872

Multi-Variable PreCoF Values:
[('school', 0.13636363636363635),
 ('Fedu', 0.07443181818181815),
 (('Fedu', 'Mjob'), 0.06988636363636364),
 (('failures', 'school'), 0.05170454545454545),
 (('Mjob', 'school'), 0.03636363636363636),
 (('age', 'school'), 0.03636363636363636),
 (('school', 'schoolsup'), 0.03636363636363636),
 (('Fedu', 'Fjob'), 0.03636363636363636),
 (('Fedu', 'failures'), 0.02840909090909091),


In [30]:
adult = fetch_data('adult')
X = adult.drop(columns=['target', 'fnlwgt', 'native-country'])
y = adult.loc[:, 'target']

cat = ['workclass','education','marital-status','occupation','relationship','race']
X[cat] = X[cat].astype('category')

precof_analysis(X, y, ['sex'], 'sex', 0, 1, 'adult')

Index(['age', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object') Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race'],
      dtype='object')
get_counterfactual_counts_descriptive took 22.6244 seconds
get_counterfactual_counts_descriptive took 36.6563 seconds
get_precof_values took 59.3097 seconds

Model without sensitive attributes
Classifier Accuracy: 0.8583

Multi-Variable PreCoF Values:
[(('capital-loss', 'education-num'), 0.19389486943572792),
 (('marital-status', 'occupation'), 0.1006464825591975),
 (('age', 'capital-loss'), 0.09990447433665157),
 (('education-num', 'marital-status'), 0.09759441355333509),
 ('marital-status', 0.08242941372327213),
 (('age', 'hours-per-week'), 0.05703646066589449),
 (('capital-loss', 'hours-per-week'), 0.03788251245728641),
 (('hours-per-week', 'marital-status'), 0.013314701417635733),
 (('marital-status', 'workclass'), 0.011981071722147848),
 (('marital-status

In [31]:
compas=pd.read_csv('DATA/cox-violent-parsed.csv')
compas['race'] = compas['race'].replace('African-American', 'African_American')
compas['race'] = compas['race'].replace('Native American', 'Native_American')

X=compas[['sex','age','age_cat', 'race', 'juv_fel_count','decile_score', 'juv_misd_count','juv_other_count', 'priors_count', 'c_charge_desc']]
y=compas['is_recid'].replace(-1, 0)

precof_analysis(X, y, ['race'], 'race', 'African_American', 1, 'compas')

Index(['age', 'juv_fel_count', 'decile_score', 'juv_misd_count',
       'juv_other_count', 'priors_count'],
      dtype='object') Index(['sex', 'age_cat', 'c_charge_desc'], dtype='object')
get_counterfactual_counts_descriptive took 20.6183 seconds
get_counterfactual_counts_descriptive took 12.6389 seconds
get_precof_values took 33.2742 seconds

Model without sensitive attributes
Classifier Accuracy: 0.8399

Multi-Variable PreCoF Values:
[(('c_charge_desc', 'juv_other_count'), 0.1530288577638252),
 (('c_charge_desc', 'juv_misd_count'), 0.11724402318280608),
 (('juv_other_count', 'priors_count'), 0.09545761893262494),
 (('decile_score', 'priors_count'), 0.0895792079207921),
 (('c_charge_desc', 'juv_fel_count'), 0.08101907751750786),
 (('age', 'juv_other_count'), 0.07528797391934317),
 (('c_charge_desc', 'priors_count'), 0.06666566046848588),
 (('juv_fel_count', 'priors_count'), 0.058307172180632694),
 (('age', 'priors_count'), 0.057534411977783195),
 (('decile_score', 'juv_misd_count'), 

In [32]:
filtered_compas = compas[
    (compas['c_charge_desc'] == 'Battery on Law Enforc Officer') 
].copy()

median_juv_other_count = compas['juv_other_count'].median()
filtered_compas['juv_other_category'] = filtered_compas['juv_other_count'].apply(
    lambda x: 'high' if x > median_juv_other_count else 'low'
)


print(filtered_compas.groupby(['race', 'juv_other_category', 'is_recid']).size().unstack(fill_value=0))

is_recid                              0   1
race             juv_other_category        
African_American high                 7   2
                 low                 20  43
Caucasian        high                 1   1
                 low                 27  15
Hispanic         high                 0   3
                 low                  5  14
Native_American  low                  3   0
Other            low                  0   3


In [33]:
dfmain = pd.read_csv("https://github.com/nkundiushuti/savry/blob/master/dat/reincidenciaJusticiaMenors.csv?raw=true",low_memory=False)
dfmain = dfmain.rename(index=str, columns={"V2_estranger": "foreigner", "V1_sexe": "sex","V60_SAVRY_total_score": "full_score", \
                                  "V115_reincidencia_2015":'recid', "V4_nacionalitat_agrupat":"national_group", \
                                  "V5_edat_fet_agrupat":"age_group","V6_provincia":"province","V9_edat_final_programa": \
                                  "age_final", "V11_antecedents":"prior_crime", "V12_nombre_ante_agrupat": \
                                   "prior_crimerec","V13_nombre_fets_agrupat": "prior_crimes", "V15_fet_agrupat": \
                                  "crime_maincat", "V16_fet_violencia": "crime_violence", "V17_fet_tipus":"crime_type", \
                                  'V68_@4_fracas_intervencions_anteriors':'past_supervision_failure','V69_@5_intents_autolesio_suicidi_anteriors':'history_self_harm',\
                                      'V70_@6_exposicio_violencia_llar':'violent_home','V71_@7_historia_maltracte_infantil':'childhood_mistreatment', 'V72_@8_delinquencia_pares':'parental_criminality',\
                                          'V74_@10_baix_rendiment_escola':'poor_school','V75_@11_delinquencia_grup_iguals':'delinquent_peer_group','V76_@12_rebuig_grup_iguals':'rejected_by_peer_group',\
                                              'V79_@15_manca_suport_personal_social':'lack_of_social_support','V80_@16_entorn_marginal':'community_disorganization'})
dfmain['label_value'] = dfmain.recid == 'Sí'
dfmain.national_group=dfmain.national_group.fillna('Spanish')
dfaequi=dfmain[['id', 'recid','label_value','full_score','foreigner','sex','national_group','age_group','province','age_final', \
            'prior_crime','prior_crimerec','prior_crimes','crime_maincat','crime_violence', 'crime_type','past_supervision_failure','history_self_harm', 'violent_home',\
                'childhood_mistreatment','parental_criminality','poor_school', 'delinquent_peer_group','rejected_by_peer_group','lack_of_social_support','community_disorganization']]

dfaequi = dfaequi[np.isfinite(dfaequi['full_score'])]
dfaequi = dfaequi.loc[dfaequi['full_score'] != 99]
dfaequi.age_group=dfaequi.age_group.fillna('16 i 17 anys')
df = dfaequi

X = df.drop(['full_score', 'id', 'label_value','recid'], axis=1)
y = (df['label_value'])
sensitive_attribute='foreigner'
sensitive_value='Estranger'

precof_analysis(X, y, ["foreigner"], "foreigner", "Estranger", True, 'catalonia')


Index(['age_final'], dtype='object') Index(['sex', 'national_group', 'age_group', 'province', 'prior_crime',
       'prior_crimerec', 'prior_crimes', 'crime_maincat', 'crime_violence',
       'crime_type', 'past_supervision_failure', 'history_self_harm',
       'violent_home', 'childhood_mistreatment', 'parental_criminality',
       'poor_school', 'delinquent_peer_group', 'rejected_by_peer_group',
       'lack_of_social_support', 'community_disorganization'],
      dtype='object')
get_counterfactual_counts_descriptive took 0.0326 seconds
get_counterfactual_counts_descriptive took 0.0490 seconds
get_precof_values took 0.0886 seconds

Model without sensitive attributes
Classifier Accuracy: 0.6732

Multi-Variable PreCoF Values:
[('age_final', 0.20606060606060606),
 ('delinquent_peer_group', 0.19797979797979798),
 (('age_final', 'prior_crimerec'), 0.11313131313131314),
 (('prior_crimerec', 'province'), 0.06868686868686869),
 (('delinquent_peer_group', 'prior_crimerec'), 0.05656565656565657

In [34]:
crime2 = pd.read_csv('DATA/fairness_dataset-main/fairness_dataset-main/experiments/data/communities_crime.csv', sep=',')
X = crime2.drop(columns=['class','communityname','state', 'ViolentCrimesPerPop', 'racepctblack', 'racePctWhite', 'racePctAsian', 'racePctHisp', 'fold','AsianPerCap','HispPerCap','whitePerCap','blackPerCap','indianPerCap', 'HispPerCap'])
y = crime2['class']

precof_analysis(X, y, ['Black'], 'Black', 1, True, 'crime2')

Index(['population', 'householdsize', 'agePct12t21', 'agePct12t29',
       'agePct16t24', 'agePct65up', 'numbUrban', 'pctUrban', 'medIncome',
       'pctWWage', 'pctWFarmSelf', 'pctWInvInc', 'pctWSocSec', 'pctWPubAsst',
       'pctWRetire', 'medFamInc', 'perCapInc', 'NumUnderPov', 'PctPopUnderPov',
       'PctLess9thGrade', 'PctNotHSGrad', 'PctBSorMore', 'PctUnemployed',
       'PctEmploy', 'PctEmplManu', 'PctEmplProfServ', 'PctOccupManu',
       'PctOccupMgmtProf', 'MalePctDivorce', 'MalePctNevMarr', 'FemalePctDiv',
       'TotalPctDiv', 'PersPerFam', 'PctFam2Par', 'PctKids2Par',
       'PctYoungKids2Par', 'PctTeen2Par', 'PctWorkMomYoungKids', 'PctWorkMom',
       'NumIlleg', 'PctIlleg', 'NumImmig', 'PctImmigRecent', 'PctImmigRec5',
       'PctImmigRec8', 'PctImmigRec10', 'PctRecentImmig', 'PctRecImmig5',
       'PctRecImmig8', 'PctRecImmig10', 'PctSpeakEnglOnly',
       'PctNotSpeakEnglWell', 'PctLargHouseFam', 'PctLargHouseOccup',
       'PersPerOccupHous', 'PersPerOwnOccHous', 'Per